In [1]:
from datasets_custom.dataset_custom import DatasetCustom
from models.models import BertModel
from preprocess.preprocess_dataset import clear_text

In [2]:
import io
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

## Leitura dos dados

In [4]:
dataset = DatasetCustom("/home/jailsonpj/Área de Trabalho/mestrado/src/siames_network_tf/exp_1/datasets_custom/")
df = dataset.get_dataset_custom()

/home/jailsonpj/Área de Trabalho/mestrado/src/siames_network_tf/exp_1/datasets_custom/dataset_custom.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_label] = pd.Categorical(df[column_label])
/home/jailsonpj/Área de Trabalho/mestrado/src/siames_network_tf/exp_1/datasets_custom/dataset_custom.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_label] = df[column_label].cat.codes


In [5]:
df.head(5)

,Source,Title,Content,labels,text
0,cnn.com,Stocks slip slightly,With little U.S. economic or corporate news on...,0,Stocks slip slightly With little U.S. economic...
1,investors.com,Stocks Down At Midday On Weak China Export Data,Stocks were down across the board at midday Mo...,1,Stocks Down At Midday On Weak China Export Dat...
2,cnn.com,Winners and losers of the bull market,It's been five years since the stock market hi...,0,Winners and losers of the bull market It's bee...
3,foxnews.com,Does Mt. Gox CEO still control all those stole...,next Image 1 of 3 prev next Image 2 of 3 prev ...,1,Does Mt. Gox CEO still control all those stole...
4,mashable.com,Bankrupt Exchange Mt. Gox Still Has $600 Milli...,Japan-based Bitcoin exchange Mt.Gox has been d...,0,Bankrupt Exchange Mt. Gox Still Has $600 Milli...


In [6]:
df = df[["text", "labels"]]

In [7]:
df.head(10)

,text,labels
0,Stocks slip slightly With little U.S. economic...,0
1,Stocks Down At Midday On Weak China Export Dat...,1
2,Winners and losers of the bull market It's bee...,0
3,Does Mt. Gox CEO still control all those stole...,1
4,Bankrupt Exchange Mt. Gox Still Has $600 Milli...,0
5,Is Bitcoin Legal? Illegal? a Currency? a Commo...,1
6,The UK's first Bitcoin digital cash machine go...,0
7,Metro-North worker dies after being hit by tra...,1
8,Democrat donor William Ackman conspires to des...,1
9,Public Transit Use In U.S. Is At a 57-Year Hig...,0


In [8]:
df = dataset.df_partition_label(df, "labels")

/home/jailsonpj/anaconda3/envs/mestrado/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
df.head(10)

,text,labels
0,Stocks slip slightly With little U.S. economic...,0
1,Stocks Down At Midday On Weak China Export Dat...,1
2,Winners and losers of the bull market It's bee...,0
3,Does Mt. Gox CEO still control all those stole...,1
4,Bankrupt Exchange Mt. Gox Still Has $600 Milli...,0
5,Is Bitcoin Legal? Illegal? a Currency? a Commo...,1
6,The UK's first Bitcoin digital cash machine go...,0
7,Metro-North worker dies after being hit by tra...,1
8,Public Transit Use In U.S. Is At a 57-Year Hig...,0
9,Democrat donor William Ackman conspires to des...,1


## Pre processamento

In [10]:
df = clear_text(df, "text")

In [11]:
df.head(5)

,text,labels
0,stocks slip slightly little economic corporate...,0
1,stocks midday weak china export data stocks bo...,1
2,winners losers bull market years stock market ...,0
3,does mt gox ceo control stolen bitcoins image ...,1
4,bankrupt exchange mt gox 600 million bitcoins ...,0


## Split Train and Test

In [12]:
train, test = train_test_split(df, test_size=0.2)

In [13]:
train.shape, test.shape

((7256, 2), (1814, 2))

### Convertendo Dataframe para tf.dataset

In [14]:
labels_train = tf.squeeze(tf.constant([train.pop("labels")]), axis=0)
labels_test = tf.squeeze(tf.constant([test.pop("labels")]), axis=0)

In [15]:
ds_train = tf.data.Dataset.from_tensor_slices((dict(train), labels_train)).batch(32)
ds_test = tf.data.Dataset.from_tensor_slices((dict(test), labels_test)).batch(32)

In [16]:
# for feat, targ in ds.take(1):
#   print ('Features: {}, Target: {}'.format(feat, targ))

## Instanciando Modelo BERT

In [17]:
bert_model = BertModel()
model = bert_model.build_model()

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

## Treino Modelo Siames com Triplet Semi Hard

In [19]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss()
)

In [20]:
# Train the network
history = model.fit(
    x=ds_train,
    epochs=100
)

Epoch 1/100
227/227 [==============================] - 923s 4s/step - loss: 0.8648
Epoch 2/100
227/227 [==============================] - 957s 4s/step - loss: 0.7020
Epoch 3/100
227/227 [==============================] - 906s 4s/step - loss: 0.6825
Epoch 4/100
227/227 [==============================] - 914s 4s/step - loss: 0.6265
Epoch 5/100
227/227 [==============================] - 896s 4s/step - loss: 0.6730
Epoch 6/100
227/227 [==============================] - 717s 3s/step - loss: 0.6007
Epoch 7/100
227/227 [==============================] - 714s 3s/step - loss: 0.5771
Epoch 8/100
227/227 [==============================] - 706s 3s/step - loss: 0.4941
Epoch 9/100
227/227 [==============================] - 725s 3s/step - loss: 0.4818
Epoch 10/100
227/227 [==============================] - 676s 3s/step - loss: 0.4071
Epoch 11/100
227/227 [==============================] - 666s 3s/step - loss: 0.4966
Epoch 12/100
227/227 [==============================] - 678s 3s/step - loss: 0.5086
E

## Resultados

In [21]:
results = model.predict(ds_test)

57/57 [==============================] - 47s 827ms/step


In [22]:
# save test embeddings for visualization in projector

np.savetxt("vecs_exp1.tsv", results, delimiter='\t')

out_m = io.open("meta_exp1.tsv", "w", encoding="utf-8")

for text, labels in tfds.as_numpy(ds_test):
    [out_m.write(str(x) + "\n") for x in labels]

out_m.close()

In [29]:
model.save("model_exp1")

INFO:tensorflow:Assets written to: model_exp1/assets


INFO:tensorflow:Assets written to: model_exp1/assets
